BLAST results processing for Rep Proteins (to get Incompatibility type) and Virulence Factors

In [ ]:
##ALTHOUGH THE CODE IS WRITTEN TO PROCESS BLAST RESULTS FOR OTHER PROTEINS, ONLY THE REP AND VIR DATA WAS USED IN THIS PUBLICATION

#Imports
import pandas as pd
import numpy as np
import datetime as dt
from tqdm import tqdm

# Only change the parameters inside here
#  -------------------------------------------------------
# Input the folder path (make sure it is r'path/')
folderPathDad = r'/workdir/users/hgl28/CDC_Isolates/PlasmidTables/'
folderPathMOB = r'/workdir/users/hgl28/CDC_Isolates/Post-Dec2022/'

# Input the name of the all of the types files (+ extension)
filenameIncT = 'rep_results/Rep_Acc_IncType.txt'
filenameMobT = 'mob_mpf_results/acc_oriTMob.txt'
filenameMPFT = 'mob_mpf_results/acc_MPF2.txt'
filenameT4CPT = 'mob_mpf_results/acc_T4CP_oriT.txt'
filenameVirT = 'virulence/VF_core_acc.txt'

# Input the name of the Blast Results files (+ extension)
filenameRGI = 'rgi_plasmidORFs.tsv'
filenameRepBR = 'rep_12-14-22.txt'
filenameMobBR = 'mob_12-14-22.txt'
filenameMPFBR = 'mpf_12-14-22.txt'
filenameT4CPBR = 'T4CP_12-14-22.txt'
filenameVirBR = 'virs_12-14-22_sed.txt'

#  Input the name of the ORF ID file (+ extension)
filenamePID = 'plasmidORF_IDs_12-14-22.txt'

# Input the Output ORF and Plasmid files (+ extension)
filenameOoutF = 'RGI_tab_12-14-22.csv'
# ------------------------------------------

# Loading the type files
IC = pd.read_csv(folderPathDad + filenameIncT, delimiter='\t', header=None)
IC.columns = ["Acc","INC"]
MOB = pd.read_csv(folderPathDad + filenameMobT, delimiter='\t', header=None)
MOB.columns = ["Acc","MOB"]
MPF = pd.read_csv(folderPathDad + filenameMPFT, delimiter='\t', header=None)
MPF.columns = ["Acc","MPF"]
T4CP = pd.read_csv(folderPathDad + filenameT4CPT, delimiter='\t', header=None)
T4CP.columns = ["Acc","T4CP"]
VIR = pd.read_csv(folderPathDad + filenameVirT, delimiter='\t', header=None)
VIR.columns = ["Acc","VIR"]
# Loading the Blast results files
RGIr = pd.read_csv(folderPathMOB + filenameRGI, delimiter='\t', header=0)
RepBR = pd.read_csv(folderPathMOB + filenameRepBR, delimiter='\t', header=None)
BRNames = RepBR.columns
RepBR = RepBR[BRNames[0:5]]
RepBR.columns = ["Query","Subject","PID","Qcov","Eval"]
MobBR = pd.read_csv(folderPathMOB + filenameMobBR, delimiter='\t', header=None)
BRNames = MobBR.columns
MobBR = MobBR[BRNames[0:5]]
MobBR.columns = ["Query","Subject","PID","Qcov","Eval"]
MPFBR = pd.read_csv(folderPathMOB + filenameMPFBR, delimiter='\t', header=None)
BRNames = MPFBR.columns
MPFBR = MPFBR[BRNames[0:5]]
MPFBR.columns = ["Query","Subject","PID","Qcov","Eval"]
T4CPBR = pd.read_csv(folderPathMOB + filenameT4CPBR, delimiter='\t', header=None)
BRNames = T4CPBR.columns
T4CPBR = T4CPBR[BRNames[0:5]]
T4CPBR.columns = ["Query","Subject","PID","Qcov","Eval"]
VirBR = pd.read_csv(folderPathMOB + filenameVirBR, delimiter='\t', header=None)
BRNames = VirBR.columns
VirBR = VirBR[BRNames[0:5]]
VirBR.columns = ["Query","Subject","PID","Qcov","Eval"]

# Loading the Plasmid ID file
PlID = pd.read_csv(folderPathMOB + filenamePID, delimiter='\t', header=0)

#Creating output file from RGI file
RGIgenes = RGIr[["ORF_ID", "Best_Hit_ARO"]].copy()
FullRGI = PlID.merge(RGIgenes, how = 'left', on = 'ORF_ID')
print(RGIgenes)
print(PlID)
print(FullRGI)
FullRGI.to_csv(folderPathMOB + filenameOoutF)
print("Antibiotic Resistance Genes Added")

# Loading output file
ORF_OF = pd.read_csv(folderPathMOB + filenameOoutF, delimiter=',', header=0)

# Setting up the Thresholds
PIDT = 50
QcovT = 50
QcovT2 = 70
EvalT = 10e-5
counter = 0
totalPlIDs = len(PlID)
RepBRIndexHit = []


#Add Inc, MOB, MPF, T4CP, and Vir Types to Blast Results tables

#Rep -----------------------------------------------------------------------------------
RepIndex = RepBR.index
for i in tqdm(RepIndex):
    RepBR.at[i, "Query"] = RepBR.loc[i].Query[:-2]
RepBRAcc = RepBR.merge(IC, how = 'inner', left_on = 'Query', right_on = 'Acc')

#MOB -----------------------------------------------------------------------------------
MobIndex = MobBR.index
for i in tqdm(MobIndex):
    MobBR.at[i, "Query"] = MobBR.loc[i].Query[:-2]
MobBRAcc = MobBR.merge(MOB, how = 'inner', left_on = 'Query', right_on = 'Acc')

#MPF -----------------------------------------------------------------------------------
MPFIndex = MPFBR.index
for i in tqdm(MPFIndex):
    MPFBR.at[i, "Query"] = MPFBR.loc[i].Query[:-2]
MPFBRAcc = MPFBR.merge(MPF, how = 'inner', left_on = 'Query', right_on = 'Acc')

#T4CP -----------------------------------------------------------------------------------
T4CPIndex = T4CPBR.index
for i in tqdm(T4CPIndex):
    T4CPBR.at[i, "Query"] = T4CPBR.loc[i].Query[:-2]
T4CPBRAcc = T4CPBR.merge(T4CP, how = 'inner', left_on = 'Query', right_on = 'Acc')

#VIR -----------------------------------------------------------------------------------
VirIndex = VirBR.index
VirBRAcc = VirBR.merge(VIR, how = 'inner', left_on = 'Query', right_on = 'Acc')
print(VirBR)
print(VirIndex)
print(VIR)
print(VirBRAcc)


#BLAST scans

#Rep -----------------------------------------------------------------------------------
for i in tqdm(PlID["ORF_ID"]):
    ChkRep = RepBRAcc.Subject == i
    if ChkRep.sum() > 0:
        tempDFRep = RepBRAcc[ChkRep]
        coverage, current_best_idx = 0, 0
        for iii in tempDFRep.index:
            tempRowRep = tempDFRep.loc[iii]
            if tempRowRep.PID >= PIDT and tempRowRep.Qcov >= QcovT and tempRowRep.Eval <= EvalT:
                if tempRowRep.Qcov >= coverage:
                    coverage = tempRowRep.Qcov
                    current_best_idx = iii
        RepBRIndexHit = np.append(RepBRIndexHit,current_best_idx)
    counter+=1
print("Rep Blast Scanning Finished! You got " + str(len(RepBRIndexHit)) + " hits out of " + str(totalPlIDs) + " ORFs.")
counter = 0
MobBRIndexHit = []
totalPlIDs
#MOB -----------------------------------------------------------------------------------
for i in tqdm(PlID["ORF_ID"]):
    ChkMOB = MobBR.Subject == i
    if ChkMOB.sum() > 0:
        tempDFMOB = MobBR[ChkMOB]
        coverage, current_best_idx = 0, 0
        for iii in tempDFMOB.index:
            tempRowMOB = tempDFMOB.loc[iii]
            if tempRowMOB.PID >= PIDT and tempRowMOB.Qcov >= QcovT2:
                if tempRowRep.Qcov >= coverage:
                    coverage = tempRowRep.Qcov
                    current_best_idx = iii
        MobBRIndexHit = np.append(MobBRIndexHit,current_best_idx)
    counter+=1
print("Mob Blast Scanning Finished! You got " + str(len(MobBRIndexHit)) + " hits out of " + str(totalPlIDs) + " ORFs.")
counter = 0
MPFBRIndexHit = []

#MPF -----------------------------------------------------------------------------------
for i in tqdm(PlID["ORF_ID"]):
    ChkMPF = MPFBR.Subject == i
    if ChkMPF.sum() > 0:
        tempDFMPF = MPFBR[ChkMPF]
        coverage, current_best_idx = 0, 0
        for iii in tempDFMPF.index:
            tempRowMPF = tempDFMPF.loc[iii]
            if tempRowMPF.PID >= PIDT and tempRowMPF.Qcov >= QcovT2:
                if tempRowRep.Qcov >= coverage:
                    coverage = tempRowRep.Qcov
                    current_best_idx = iii
        MPFBRIndexHit = np.append(MPFBRIndexHit,current_best_idx)
    counter+=1
print("MPF Blast Scanning Finished! You got " + str(len(MPFBRIndexHit)) + " hits out of " + str(totalPlIDs) + " ORFs.")
counter = 0
T4CPBRIndexHit = []

#T4CP -----------------------------------------------------------------------------------
for i in tqdm(PlID["ORF_ID"]):
    ChkT4CP = T4CPBR.Subject == i
    if ChkT4CP.sum() > 0:
        tempDFT4CP = T4CPBR[ChkT4CP]
        coverage, current_best_idx = 0, 0
        for iii in tempDFT4CP.index:
            tempRowT4CP = tempDFT4CP.loc[iii]
            if tempRowT4CP.PID >= PIDT and tempRowT4CP.Qcov >= QcovT2:
                if tempRowRep.Qcov >= coverage:
                    coverage = tempRowRep.Qcov
                    current_best_idx = iii
        T4CPBRIndexHit = np.append(T4CPBRIndexHit,current_best_idx)
    counter+=1
print("T4CP Blast Scanning Finished! You got " + str(len(T4CPBRIndexHit)) + " hits out of " + str(totalPlIDs) + " ORFs.")
counter = 0
VirBRIndexHit = []

#VIR -----------------------------------------------------------------------------------
for i in tqdm(PlID["ORF_ID"]):
    ChkVIR = VirBR.Subject == i
    if ChkVIR.sum() > 0:
        tempDFVIR = VirBR[ChkVIR]
        coverage, current_best_idx = 0, 0
        for iii in tempDFVIR.index:
            tempRowVIR = tempDFVIR.loc[iii]
            if tempRowVIR.PID >= PIDT and tempRowVIR.Qcov >= QcovT2 and tempRowRep.Eval <= EvalT:
                if tempRowRep.Qcov >= coverage:
                    coverage = tempRowRep.Qcov
                    current_best_idx = iii
        VirBRIndexHit = np.append(VirBRIndexHit,current_best_idx)
    counter+=1
print("Vir Blast Scanning Finished! You got " + str(len(VirBRIndexHit)) + " hits out of " + str(totalPlIDs) + " ORFs.")



# Scan Hits and add to table

#Rep -----------------------------------------------------------------------------------
RepHits = RepBRAcc.iloc[RepBRIndexHit] 
RepHitsmall = RepHits[["Subject", "INC"]].copy().drop_duplicates()
RepOF = ORF_OF.merge(RepHitsmall, how='left', left_on='ORF_ID',right_on='Subject')
RepOF = RepOF[["ORF_ID", "Best_Hit_ARO", "INC"]].copy()

now = dt.datetime.now()
dt_string = now.strftime("%Y-%m-%d_%H-%M-%S")
RepOF.to_csv(folderPathMOB + "Inc_Tab_"+ dt_string + ".csv")
print("Inc Type Added ", len(RepOF))

#MOB -----------------------------------------------------------------------------------
MobHits = MobBRAcc.iloc[MobBRIndexHit]
MobHitsmall = MobHits[["Subject", "MOB"]].copy().drop_duplicates()
MobOF = ORF_OF.merge(MobHitsmall, how='left', left_on='ORF_ID',right_on='Subject')
MobOF = MobOF[["ORF_ID", "Best_Hit_ARO", "MOB"]].copy()

now = dt.datetime.now()
dt_string = now.strftime("%Y-%m-%d_%H-%M-%S")
MobOF.to_csv(folderPathMOB + "Mob_Tab_"+ dt_string + ".csv")
print("MOB Type Added ", len(MobOF))

#MPF -----------------------------------------------------------------------------------
MPFHits = MPFBRAcc.iloc[MPFBRIndexHit]
MPFHitsmall = MPFHits[["Subject", "MPF"]].copy().drop_duplicates()
MPFOF = ORF_OF.merge(MPFHitsmall, how='left', left_on='ORF_ID',right_on='Subject')
MPFOF = MPFOF[["ORF_ID", "Best_Hit_ARO", "MPF"]].copy()

now = dt.datetime.now()
dt_string = now.strftime("%Y-%m-%d_%H-%M-%S")
MPFOF.to_csv(folderPathMOB + "MPF_Tab_"+ dt_string + ".csv")
print("MPF Type Added ", len(MPFOF))

#T4CP -----------------------------------------------------------------------------------
T4CPHits = T4CPBRAcc.iloc[T4CPBRIndexHit]
T4CPHitsmall = T4CPHits[["Subject", "T4CP"]].copy().drop_duplicates()
T4CPOF = ORF_OF.merge(T4CPHitsmall, how='left', left_on='ORF_ID',right_on='Subject')
T4CPOF = T4CPOF[["ORF_ID", "Best_Hit_ARO", "T4CP"]].copy()

now = dt.datetime.now()
dt_string = now.strftime("%Y-%m-%d_%H-%M-%S")
T4CPOF.to_csv(folderPathMOB + "T4CP_Tab_"+ dt_string + ".csv")
print("T4CP Genes Added ", len(T4CPOF))

#VIR -----------------------------------------------------------------------------------
VirHits = VirBRAcc.iloc[VirBRIndexHit]
VirHitsmall = VirHits[["Subject", "VIR"]].copy().drop_duplicates()
VirOF = ORF_OF.merge(VirHitsmall, how='left', left_on='ORF_ID',right_on='Subject')
VirOF = VirOF[["ORF_ID", "Best_Hit_ARO", "VIR"]].copy()

now = dt.datetime.now()
dt_string = now.strftime("%Y-%m-%d_%H-%M-%S")
VirOF.to_csv(folderPathMOB + "VIR_Tab_"+ dt_string + ".csv")
print("Virulence Genes Added ", len(VirOF))

#Making the final output file
RepFinal = RepOF[["ORF_ID", "Best_Hit_ARO", "INC"]].copy().drop_duplicates()
MOBFinal = MobOF[["ORF_ID","MOB"]].copy().drop_duplicates()
MPFFinal = MPFOF[["ORF_ID","MPF"]].copy().drop_duplicates()
T4CPFinal = T4CPOF[["ORF_ID","T4CP"]].copy().drop_duplicates()
VIRFinal = VirOF[["ORF_ID","VIR"]].copy().drop_duplicates()

Merge = RepFinal.merge(MOBFinal, how='outer', on='ORF_ID')
Merge = Merge.merge(MPFFinal,  how='outer', on='ORF_ID')
Merge = Merge.merge(T4CPFinal,  how='outer', on='ORF_ID')
FinalOF = Merge.merge(VIRFinal,  how='outer', on='ORF_ID').drop_duplicates()

now = dt.datetime.now()
dt_string = now.strftime("%Y-%m-%d_%H-%M-%S")
FinalOF.to_csv(folderPathMOB + "Final_ORF_tab_" + dt_string + ".csv")
print("Everything Finished ", len(FinalOF))

Adding MOBtyper and RGI results to plasmid table

In [ ]:
#Imports
from tqdm import tqdm
import pandas as pd
import numpy as np

#File and Folder names (due to changes made outside of python code, date numbers in names might not match with other cells)
FolderPath = r"/workdir/users/hgl28/CDC_Isolates/Post-Dec2022/"
MD_name = "metadata_01-12-23.csv"
Plastab_name = "Plasmid_tab2_01-09-23.csv"
mobFile = "12-14-22_MobTyperResults_sed.txt"
filenameRGI = 'Plasmid_Tab_withHits_2023-01-09_sed.csv'


#Read files
Plastab = pd.read_csv(FolderPath + Plastab_name, delimiter=",", header=0)
MDa = pd.read_csv(FolderPath + MD_name, delimiter=",", header=0)
mobtypertab = pd.read_csv(FolderPath + mobFile, delimiter="\t", header=0)
mobtypertab_short = mobtypertab[["sample_id", "rep_type(s)", "relaxase_type(s)", "mpf_type", "orit_type(s)", "predicted_mobility", "predicted_host_range_overall_rank"]].copy()
RGIr = pd.read_csv(FolderPath + filenameRGI, delimiter=',', header=0)

#Adding ARG data
RGIgenes = RGIr[["Plasmid_ID", "ARGs", "Virulence"]].copy()
FullRGI = Plastab.merge(RGIgenes, how = 'left', on = 'Plasmid_ID')
Plastab = FullRGI[["Plasmid_ID", "Length", "GC", "ARGs", "Virulence"]].copy()
print("Antibiotic Resistance and Virulence Genes Added")

#Merging MOB-typer data with plasmid table
Plastab_Merge = Plastab.merge(mobtypertab_short, how='left', left_on='Plasmid_ID', right_on='sample_id')
Plastab_clean = Plastab_Merge[["Plasmid_ID", "Length", "GC", "ARGs", "Virulence", "rep_type(s)", "relaxase_type(s)", "mpf_type", "orit_type(s)", "predicted_mobility", "predicted_host_range_overall_rank"]].copy()
print("MOB-typer data added")

#Add Metadata to Plasmid Table
MDalist = []
for i in MDa["Acc"]:
    MDalist.append(i)

append = ".1"
MDAlist_suf = [suf + append for suf in MDalist]
counter = 0
for ii in MDa.index:
    MDa.loc[ii, "Acc"] = MDAlist_suf[counter]
    counter+=1

Merged = Plastab_clean.merge(MDa, how='left', right_on='Acc', left_on='Plasmid_ID', copy=False)
Merged = Merged[["Genus", "Organism", "AR_Bank_ID", "Plasmid_ID", "Length", "GC", "ARGs", "rep_type(s)", "relaxase_type(s)", "mpf_type", "orit_type(s)", "predicted_mobility", "predicted_host_range_overall_rank", "Virulence"]].copy()
print("Isolate data added")
Merged.to_csv(FolderPath + "PlasmidTab_mob_withMDa_01-12-23.csv")


Adding rmsFinder results to plasmid table

In [ ]:
#Imports
import pandas as pd
import numpy as np
import datetime as dt
from tqdm import tqdm

# Input the folder path (make sure it is r'path/')
folderPath = r'/workdir/users/hgl28/CDC_Isolates/Post-Dec2022/'
# Filenames (due to changes made outside of python code, date numbers in names might not match with other cells)
filenameRE = 'rmsFinder/plasmid_ORFs_Rmsfinder_RE.csv'
filenameMT = 'rmsFinder/plasmid_ORFs_Rmsfinder_MT.csv'
filenamePID = 'plasmidORF_IDs_12-14-22.txt'
filenamePLid = 'plasmidIDs_01-23-23.txt'
filenamePlOF = 'Plasmid_Tab_sed_01-23-23.csv'
filenameORF_OF = 'InterPlasTabs/RGI_tab_12-14-22.csv'

# Loading the Blast results files
REtab = pd.read_csv(folderPath + filenameRE, delimiter=',', header=0)
MTtab = pd.read_csv(folderPath + filenameMT, delimiter=',', header=0)

#Loading Output Files
ORF_OF = pd.read_csv(folderPath + filenameORF_OF, delimiter=',', header=0)
Pl_OF = pd.read_csv(folderPath + filenamePlOF, delimiter=',', header=0)

# Loading the Plasmid ID files
PlID = pd.read_csv(folderPath + filenamePID, delimiter='\t', header=0)
PLasmidID = pd.read_csv(folderPath + filenamePLid, delimiter='\t', header=0)
PLasmidID = PLasmidID.drop_duplicates()

# Setting up the Thresholds
PIDT = 50
EvalT = 10e-5
counter = 0
totalPlIDs = len(PlID)
PlasmidIDS = len(PLasmidID)
REIndexHit = []
MTIndexHit = []

#rmsFinder Results scans

#Restriction Enzymes -----------------------------------------------------------------------------------
for i in tqdm(PlID["ORF_ID"]):
    ChkRE = REtab.qseqid == i
    if ChkRE.sum() > 0:
        tempDFRE = REtab[ChkRE]
        current_best_idx = 0
        for iii in tempDFRE.index:
            tempRowTA = tempDFRE.loc[iii]
            if tempRowTA.pident >= PIDT and tempRowTA.evalue <= EvalT and tempRowTA.coverage_threshold_met == True:
                current_best_idx = iii
        REIndexHit = np.append(REIndexHit,current_best_idx)
    counter+=1
print("RE rmsFinder results Scanning Finished! You got " + str(len(REIndexHit)) + " hits out of " + str(totalPlIDs) + " ORFs.")
counter = 0

#Methylases -----------------------------------------------------------------------------------
for i in tqdm(PlID["ORF_ID"]):
    ChkMT = MTtab.qseqid == i
    if ChkMT.sum() > 0:
        tempDFMT = MTtab[ChkMT]
        current_best_idx = 0
        for iii in tempDFMT.index:
            tempRowTA = tempDFMT.loc[iii]
            if tempRowTA.pident >= PIDT and tempRowTA.evalue <= EvalT and tempRowTA.coverage_threshold_met == True:
                current_best_idx = iii
        MTIndexHit = np.append(MTIndexHit,current_best_idx)
    counter+=1
print("MT rmsFinder results Scanning Finished! You got " + str(len(MTIndexHit)) + " hits out of " + str(totalPlIDs) + " ORFs.")
counter = 0


# Scan Hits and add to table
#Restriction Enzymes -----------------------------------------------------------------------------------
REHits = REtab.iloc[REIndexHit] 
Re_OF = ORF_OF.merge(REHits, how='left', left_on='ORF_ID',right_on='qseqid')
ReOF = Re_OF[["ORF_ID", "Best_Hit_ARO", "sseqid"]].copy()
dummytabname = 'Plasmid_tab2_01-09-23.csv'
dummytab = pd.read_csv(folderPath + dummytabname, delimiter=',', header=0)
for i in tqdm(dummytab["Plasmid_ID"]):
    index = dummytab[dummytab.Plasmid_ID == i]
    Chk = ReOF["ORF_ID"].str.contains(i)
    match = ReOF[Chk]
    for ii in match.index:
        temprow = match.loc[ii]
        #TA
        NewARG = str(temprow[2])
        tempstr = dummytab.at[dummytab[dummytab.Plasmid_ID == i].index[0],"Restriction Enzymes"]
        temptype = type(dummytab.at[dummytab[dummytab.Plasmid_ID == i].index[0],"Restriction Enzymes"])
        if temptype == str:
            dummytab.at[dummytab[dummytab.Plasmid_ID == i].index[0],"Restriction Enzymes"] = tempstr + " " + NewARG
        else:
            dummytab.at[dummytab[dummytab.Plasmid_ID == i].index[0],"Restriction Enzymes"] = NewARG

#Methylases -----------------------------------------------------------------------------------
MTHits = MTtab.iloc[REIndexHit] 
Mt_OF = ORF_OF.merge(MTHits, how='left', left_on='ORF_ID',right_on='qseqid')
MtOF = Mt_OF[["ORF_ID", "Best_Hit_ARO", "sseqid"]].copy()
dummytabname = 'Plasmid_tab3_01-09-23.csv'
dummytab2 = pd.read_csv(folderPath + dummytabname, delimiter=',', header=0)
for i in tqdm(dummytab2["Plasmid_ID"]):
    index = dummytab2[dummytab2.Plasmid_ID == i]
    Chk = MtOF["ORF_ID"].str.contains(i)
    match = MtOF[Chk]
    for ii in match.index:
        temprow = match.loc[ii]
        #TA
        NewARG = str(temprow[2])
        tempstr = dummytab2.at[dummytab2[dummytab2.Plasmid_ID == i].index[0],"Methylases"]
        temptype = type(dummytab2.at[dummytab2[dummytab2.Plasmid_ID == i].index[0],"Methylases"])
        if temptype == str:
            dummytab2.at[dummytab2[dummytab2.Plasmid_ID == i].index[0],"Methylases"] = tempstr + " " + NewARG
        else:
            dummytab2.at[dummytab2[dummytab2.Plasmid_ID == i].index[0],"Methylases"] = NewARG

#Final tables and finish
Plastab_0123 = dummytab2.merge(dummytab, how='left', on='Plasmid_ID')
Plastab_012323 = Pl_OF.merge(Plastab_0123, how='left', on='Plasmid_ID')
Plastab_012323 = Plastab_012323[["Genus","Organism","AR_Bank_ID","Plasmid_ID","Length","GC","ARGs","rep_type(s)","relaxase_type(s)","mpf_type","orit_type(s)","predicted_mobility","predicted_host_range_overall_rank","Virulence", "Protein T-AT", "RNA T-AT", "Restriction Enzymes", "Methylases"]].copy()
Plastab_012323.to_csv(folderPath + 'Plasmid_Tab_full_01-31-23.csv')
print("All Done!")

Getting plasmid metadata (downloaded from NCBI)

In [ ]:
#Getting Plasmid Metadata from XML file

from tqdm import tqdm
import pandas as pd
import numpy as np
import xml.etree.cElementTree as ET

xmlPath = r"/workdir/users/hgl28/CDC_Isolates/Post-Dec2022/Plasmid_Info_NCBI.xml"
FolderPath = r"/workdir/users/hgl28/CDC_Isolates/Post-Dec2022/"

tree = ET.parse(xmlPath)
root = tree.getroot()
print(root.tag, root.attrib)

Acctab = []
Nametab = []
Genustab = []
ARbanktab = []

for Acc in root.iter('Textseq-id_accession'):
    Acctab.append(Acc.text)
print(len(Acctab))

for Name in root.iter('Org-ref_taxname'):
    Nametab.append(Name.text)
print(len(Nametab))

for Genus in root.iter("BinomialOrgName_genus"):
    Genustab.append(Genus.text)
print(len(Genustab))

for mod in root.iter("OrgMod"):
    for subtype in mod.findall("OrgMod_subtype"):
        if subtype.text == '2':
            for ARbankID in mod.findall("OrgMod_subname"):
                ARbanktab.append(ARbankID.text)
print(len(ARbanktab))

MD_df = pd.DataFrame(list(zip(Acctab, Genustab, Nametab, ARbanktab)), columns=['Acc', 'Genus', 'Organism', 'AR_Bank_ID'])
MD_df.to_csv(FolderPath + "metadata_01-12-23.csv")

#Adding said plasmid metadata to the plasmid table

FolderPath = r"/workdir/users/hgl28/CDC_Isolates/Post-Dec2022/"
MD_name = "metadata_01-10-23.csv"
Plastab_name = "Plasmid_Tab_withHits_2023-01-09_sed.csv"

Plastab = pd.read_csv(FolderPath + Plastab_name, delimiter=",", header=0)
MDa = pd.read_csv(FolderPath + MD_name, delimiter=",", header=0)

#Add Metadata to Plasmid Table
MDalist = []
for i in MDa["Acc"]:
    MDalist.append(i)

append = ".1"
MDAlist_suf = [suf + append for suf in MDalist]
counter = 0
for ii in MDa.index:
    MDa.loc[ii, "Acc"] = MDAlist_suf[counter]
    counter+=1

Merged = Plastab.merge(MDa, how='left', right_on='Acc', left_on='Plasmid_ID', copy=False)
Merged = Merged[["Organism","AR_Bank_ID","Plasmid_ID","Length","GC","ARGs","Inc_Type","MOB_Type","MPF_Type","T4CP","Virulence"]].copy()
Merged.to_csv(FolderPath + "PlasmidTab_withMDa_01-10-23.csv")

BLAST results processing for Partitioning Proteins

In [ ]:
#Blast Scanning for Par Proteins

#Re running imports and file loading
import pandas as pd
import numpy as np
import datetime as dt
from tqdm import tqdm

# Input the folder path (make sure it is r'path/') and filenames (due to changes made outside of python code, date numbers in names might not match with other cells)
folderPath = r'/workdir/users/hgl28/CDC_Isolates/Post-Dec2022/'
filenamePID = 'plasmidORF_IDs_12-14-22.txt'
filenamePLid = 'plasmidIDs_01-23-23.txt'
filenamePlOF = 'Plasmid_Tab_sed_01-23-23.csv'
filenameORF_OF = 'InterPlasTabs/RGI_tab_12-14-22.csv'
filename_parBR = 'Partitioning/all_pars_sed_02-02-23.txt'


#Read Blast Results
parBR = pd.read_csv(folderPath + filename_parBR, delimiter='\t', header=None)
parBRNames = parBR.columns
parBR = parBR[parBRNames[0:5]]
parBR.columns = ["Query","Subject","PID","Qcov","Eval"]

# Setting up the Thresholds
PIDT = 80
QcovT2 = 70
EvalT = 10e-5
counter = 0
totalPlIDs = len(PlID)
ParBRIndexHit = []

#Add Par Protein Families to Blast Results tables

#--------------------------------------------------------------------------------------
ParAcc = pd.read_csv(folderPath + 'Partitioning/all_pars_titles.csv', delimiter=',',header=0)
parBR = parBR.merge(ParAcc, how = 'inner', left_on = 'Query', right_on = 'ACC')

#--------------------------------------------------------------------------------------
for i in tqdm(PlID["ORF_ID"]):
    ChkPar = parBR.Subject == i
    if ChkPar.sum() > 0:
        tempDFPar = parBR[ChkPar]
        coverage, current_best_idx = 0, 0
        for iii in tempDFPar.index:
            tempRowPar = tempDFPar.loc[iii]
            if tempRowPar.PID >= PIDT and tempRowPar.Qcov >= QcovT2 and tempRowPar.Eval <= EvalT:
                if tempRowPar.Qcov >= coverage:
                    coverage = tempRowPar.Qcov
                    current_best_idx = iii
        ParBRIndexHit = np.append(ParBRIndexHit,current_best_idx)
    counter+=1
print("Partition Protein Blast Scanning Finished! You got " + str(len(ParBRIndexHit)) + " hits out of " + str(totalPlIDs) + " ORFs.")
counter = 0

# Scan Hits and add to table

#--------------------------------------------------------------------------------------
ParHits = parBR.iloc[ParBRIndexHit] 
ParOF = ORF_OF.merge(ParHits, how='left', left_on='ORF_ID',right_on='Subject')
ParOF = ParOF[["ORF_ID", "PAR"]].copy().drop_duplicates()
print("Par Proteins Added ", len(ParOF))
now = dt.datetime.now()
dt_string = now.strftime("%Y-%m-%d_%H-%M-%S")
ParOF.to_csv(folderPath + "Par_Tab_"+ dt_string + ".csv")
print("All done!")

#Adding stuff to plasmid table
PlastabOF = pd.read_csv(folderPath + "sed_PlasmidTab_02-01-23.csv", delimiter=',', header=0)
Plastab = pd.read_csv(folderPath + 'Plasmid_tab2_01-09-23.csv', delimiter=',', header=0)
ORFtab = ParOF.dropna(subset=["PAR"])
#Identify plasmid ORFs and replace empty cells with hits from ORFs in plasmid table
for i in tqdm(PLasmidID["Plasmid_ID"]):
    index = Plastab[Plastab.Plasmid_ID == i]
    Chk = ORFtab["ORF_ID"].str.contains(i)
    match = ORFtab[Chk]
    for ii in match.index:
        temprow = match.loc[ii]
        #ARGs
        NewARG = str(temprow[1])
        tempstr = Plastab.at[Plastab[Plastab.Plasmid_ID == i].index[0],"Par"]
        temptype = type(Plastab.at[Plastab[Plastab.Plasmid_ID == i].index[0],"Par"])
        if temptype == str:
            Plastab.at[Plastab[Plastab.Plasmid_ID == i].index[0],"Par"] = tempstr + " " + NewARG
        else:
            Plastab.at[Plastab[Plastab.Plasmid_ID == i].index[0],"Par"] = NewARG


#Finishing touches and output    
print("Everything Finished")
now = dt.datetime.now()
dt_string = now.strftime("%Y-%m-%d_%H-%M-%S")

Plastab.to_csv(folderPath + "Plasmid_TabPar_withHits_"+ dt_string + ".csv")

Full_Plastab = pd.read_csv(folderPath + "sed_PlasmidTab_02-01-23.csv", delimiter=',', header=0)
Plastab = pd.read_csv(folderPath + 'Plasmid_TabPar_withHits_2023-02-02_17-01-19.csv', delimiter=',', header=0)
Plastab.drop(columns=["Length", "GC"], inplace=True)
Full_merge = Full_Plastab.merge(Plastab, how='left', on=["Plasmid_ID"])
Full_mergeOF = Full_merge[["Genus","Organism","AR_Bank_ID","Plasmid_ID","Length","GC","ARGs","rep_type(s)","relaxase_type(s)","mpf_type","orit_type(s)","predicted_mobility","predicted_host_range_overall_rank","Virulence","Protein T-AT","RNA T-AT","Restriction Enzymes","Methylases","Par"]]
Full_mergeOF.to_csv(folderPath + 'Plasmid_Tab_Full_02-06-23.csv')

BLAST results processing for Plasmid SOS Inhibition Proteins

In [ ]:
#Repeating Partitioning code for Plasmid SOS Inhibition
#Blast Scanning for Psi Proteins
#Re running imports and file loading
import pandas as pd
import numpy as np
import datetime as dt
from tqdm import tqdm

# Input the folder path (make sure it is r'path/') and filenames (due to changes made outside of python code, date numbers in names might not match with other cells)
folderPath = r'/workdir/users/hgl28/CDC_Isolates/Post-Dec2022/'
filename_psiBR = 'PlasmidSOS/psis_02-06-23.txt'
filenamePID = 'plasmidORF_IDs_12-14-22.txt'
filenamePLid = 'plasmidIDs_01-23-23.txt'
filenamePlOF = 'Plasmid_Tab_Full_02-06-23.csv'
filenameORF_OF = 'InterPlasTabs/RGI_tab_12-14-22.csv'

# Loading the Blast results files
psiBR = pd.read_csv(folderPath + filename_psiBR, delimiter='\t', header=None)
psiBRNames = psiBR.columns
psiBR = psiBR[psiBRNames[0:5]]
psiBR.columns = ["Query","Subject","PID","Qcov","Eval"]

#Loading Output Files
ORF_OF = pd.read_csv(folderPath + filenameORF_OF, delimiter=',', header=0)
Pl_OF = pd.read_csv(folderPath + filenamePlOF, delimiter=',', header=0)

# Loading the Plasmid ID files
PlID = pd.read_csv(folderPath + filenamePID, delimiter='\t', header=0)
PLasmidID = pd.read_csv(folderPath + filenamePLid, delimiter='\t', header=0)
PLasmidID = PLasmidID.drop_duplicates()

# Setting up the Thresholds
PIDT = 80
QcovT2 = 70
EvalT = 10e-5
counter = 0
totalPlIDs = len(PlID)
PsiBRIndexHit = []


#Add Psi Protein Families to Blast Results tables

#--------------------------------------------------------------------------------------
PsiAcc = pd.read_csv(folderPath + 'PlasmidSOS/allpsis_titles.csv', delimiter=',',header=0)
psiBR = psiBR.merge(PsiAcc, how = 'inner', left_on = 'Query', right_on = 'ACC')

#--------------------------------------------------------------------------------------
for i in tqdm(PlID["ORF_ID"]):
    ChkPsi = psiBR.Subject == i
    if ChkPsi.sum() > 0:
        tempDFPar = psiBR[ChkPsi]
        coverage, current_best_idx = 0, 0
        for iii in tempDFPar.index:
            tempRowPar = tempDFPar.loc[iii]
            if tempRowPar.PID >= PIDT and tempRowPar.Qcov >= QcovT2 and tempRowPar.Eval <= EvalT:
                if tempRowPar.Qcov >= coverage:
                    coverage = tempRowPar.Qcov
                    current_best_idx = iii
        PsiBRIndexHit = np.append(PsiBRIndexHit,current_best_idx)
    counter+=1
print("Plasmid SOS Inhibition Protein Blast Scanning Finished! You got " + str(len(PsiBRIndexHit)) + " hits out of " + str(totalPlIDs) + " ORFs.")
counter = 0

# Scan Hits and add to table

#--------------------------------------------------------------------------------------
PsiHits = psiBR.iloc[PsiBRIndexHit] 
PsiOF = ORF_OF.merge(PsiHits, how='left', left_on='ORF_ID',right_on='Subject')
PsiOF = PsiOF[["ORF_ID", "PSI"]].copy().drop_duplicates()
print("Psi Proteins Added ", len(PsiOF))
now = dt.datetime.now()
dt_string = now.strftime("%Y-%m-%d_%H-%M-%S")
PsiOF.to_csv(folderPath + "Psi_Tab_"+ dt_string + ".csv")
print("All done!")


#Adding stuff to plasmid table
Plastab = pd.read_csv(folderPath + 'Plasmid_tab2_01-09-23.csv', delimiter=',', header=0)
ORFtab = PsiOF.dropna(subset=["PSI"])
#Identify plasmid ORFs and replace empty cells with hits from ORFs in plasmid table
for i in tqdm(PLasmidID["Plasmid_ID"]):
    index = Plastab[Plastab.Plasmid_ID == i]
    Chk = ORFtab["ORF_ID"].str.contains(i)
    match = ORFtab[Chk]
    for ii in match.index:
        temprow = match.loc[ii]
        #ARGs
        NewARG = str(temprow[1])
        tempstr = Plastab.at[Plastab[Plastab.Plasmid_ID == i].index[0],"Psi"]
        temptype = type(Plastab.at[Plastab[Plastab.Plasmid_ID == i].index[0],"Psi"])
        if temptype == str:
            Plastab.at[Plastab[Plastab.Plasmid_ID == i].index[0],"Psi"] = tempstr + " " + NewARG
        else:
            Plastab.at[Plastab[Plastab.Plasmid_ID == i].index[0],"Psi"] = NewARG


#Finishing touches and output  
Plastab.drop(columns=["Length", "GC"], inplace=True)  
Psi_Final = Pl_OF.merge(Plastab, how='left', on='Plasmid_ID')
Psi_Final = Psi_Final[["Genus","Organism","AR_Bank_ID","Plasmid_ID","Length","GC","ARGs","rep_type(s)","relaxase_type(s)","mpf_type","orit_type(s)","predicted_mobility","predicted_host_range_overall_rank","Virulence","Restriction Enzymes","Methylases","Par","Toxin","Antitoxin","Psi"]].copy().drop_duplicates()
now = dt.datetime.now()
dt_string = now.strftime("%Y-%m-%d_%H-%M-%S")
Psi_Final.to_csv(folderPath + "Plasmid_TabPar_withHits_"+ dt_string + ".csv")
print("Everything Finished")

BLAST results processing for Mating-Pair Stabilization Proteins

In [ ]:
#Rerunning BLAST code for MPS (really should have made this a callable function)
#Re running imports and file loading
import pandas as pd
import numpy as np
import datetime as dt
from tqdm import tqdm

# Input the folder path (make sure it is r'path/') and filenames (due to changes made outside of python code, date numbers in names might not match with other cells)
folderPath = r'/workdir/users/hgl28/CDC_Isolates/Post-Dec2022/'
filename_psiBR = 'MPS/mps_02-07-23.txt'
filenamePID = 'plasmidORF_IDs_12-14-22.txt'
filenamePLid = 'plasmidIDs_01-23-23.txt'
filenamePlOF = 'Plasmid_TabPsi_withHits_2023-02-06_19-32-56.csv'
filenameORF_OF = 'InterPlasTabs/RGI_tab_12-14-22.csv'

# Loading the Blast results files
psiBR = pd.read_csv(folderPath + filename_psiBR, delimiter='\t', header=None)
psiBRNames = psiBR.columns
psiBR = psiBR[psiBRNames[0:5]]
psiBR.columns = ["Query","Subject","PID","Qcov","Eval"]

#Loading Output Files
ORF_OF = pd.read_csv(folderPath + filenameORF_OF, delimiter=',', header=0)
Pl_OF = pd.read_csv(folderPath + filenamePlOF, delimiter=',', header=0)

# Loading the Plasmid ID files
PlID = pd.read_csv(folderPath + filenamePID, delimiter='\t', header=0)
PLasmidID = pd.read_csv(folderPath + filenamePLid, delimiter='\t', header=0)
PLasmidID = PLasmidID.drop_duplicates()

# Setting up the Thresholds
PIDT = 80
QcovT2 = 70
EvalT = 10e-5
counter = 0
totalPlIDs = len(PlID)
PsiBRIndexHit = []


#Add Psi Protein Families to Blast Results tables

#--------------------------------------------------------------------------------------
PsiAcc = pd.read_csv(folderPath + 'MPS/all_MPS_titles.csv', delimiter=',',header=0)
psiBR = psiBR.merge(PsiAcc, how = 'inner', left_on = 'Query', right_on = 'ACC')

#--------------------------------------------------------------------------------------
for i in tqdm(PlID["ORF_ID"]):
    ChkPsi = psiBR.Subject == i
    if ChkPsi.sum() > 0:
        tempDFPar = psiBR[ChkPsi]
        coverage, current_best_idx = 0, 0
        for iii in tempDFPar.index:
            tempRowPar = tempDFPar.loc[iii]
            if tempRowPar.PID >= PIDT and tempRowPar.Qcov >= QcovT2 and tempRowPar.Eval <= EvalT:
                if tempRowPar.Qcov >= coverage:
                    coverage = tempRowPar.Qcov
                    current_best_idx = iii
        PsiBRIndexHit = np.append(PsiBRIndexHit,current_best_idx)
    counter+=1
print("MPS Protein Blast Scanning Finished! You got " + str(len(PsiBRIndexHit)) + " hits out of " + str(totalPlIDs) + " ORFs.")
counter = 0

# Scan Hits and add to table

#--------------------------------------------------------------------------------------
PsiHits = psiBR.iloc[PsiBRIndexHit] 
PsiOF = ORF_OF.merge(PsiHits, how='left', left_on='ORF_ID',right_on='Subject')
PsiOF = PsiOF[["ORF_ID", "MPS"]].copy().drop_duplicates()
print("MPS Proteins Added ", len(PsiOF))
now = dt.datetime.now()
dt_string = now.strftime("%Y-%m-%d_%H-%M-%S")
PsiOF.to_csv(folderPath + "MPS_Tab_"+ dt_string + ".csv")
print("All done!")


#Adding stuff to plasmid table
Plastab = pd.read_csv(folderPath + 'Plasmid_tab2_01-09-23.csv', delimiter=',', header=0)
ORFtab = PsiOF.dropna(subset=["MPS"])
#Identify plasmid ORFs and replace empty cells with hits from ORFs in plasmid table
for i in tqdm(PLasmidID["Plasmid_ID"]):
    index = Plastab[Plastab.Plasmid_ID == i]
    Chk = ORFtab["ORF_ID"].str.contains(i)
    match = ORFtab[Chk]
    for ii in match.index:
        temprow = match.loc[ii]
        #ARGs
        NewARG = str(temprow[1])
        tempstr = Plastab.at[Plastab[Plastab.Plasmid_ID == i].index[0],"MPS"]
        temptype = type(Plastab.at[Plastab[Plastab.Plasmid_ID == i].index[0],"MPS"])
        if temptype == str:
            Plastab.at[Plastab[Plastab.Plasmid_ID == i].index[0],"MPS"] = tempstr + " " + NewARG
        else:
            Plastab.at[Plastab[Plastab.Plasmid_ID == i].index[0],"MPS"] = NewARG


#Finishing touches and output  
Plastab.drop(columns=["Length", "GC"], inplace=True)  
Psi_Final = Pl_OF.merge(Plastab, how='left', on='Plasmid_ID')
Psi_Final = Psi_Final[["Genus","Organism","AR_Bank_ID","Plasmid_ID","Length","GC","ARGs","rep_type(s)","relaxase_type(s)","mpf_type","orit_type(s)","predicted_mobility","predicted_host_range_overall_rank","Virulence","Restriction Enzymes","Methylases","Par","Toxin","Antitoxin","Psi","MPS"]].copy().drop_duplicates()
now = dt.datetime.now()
dt_string = now.strftime("%Y-%m-%d_%H-%M-%S")
Psi_Final.to_csv(folderPath + "Plasmid_TabMPS_withHits_"+ dt_string + ".csv")
print("Everything Finished")

Making Figure 1

In [ ]:
#Importing
import pandas as pd
import numpy as np
import datetime as dt
from tqdm import tqdm
from plotnine import *

# Input the folder path (make sure it is r'path/') and filenames (due to changes made outside of python code, date numbers in names might not match with other cells)
folderPath = r'/workdir/users/hgl28/CDC_Isolates/Post-Dec2022/'
Plasfile = 'Plasmid_TabMPS_withHits_2023-02-07_14-11-15.csv'
Plastab = pd.read_csv(folderPath + Plasfile, delimiter=',', header=0)

#Making length into kb instead of bp
Temptab = Plastab
for index in Temptab.index:
    length = Temptab["Length"].loc[index]
    Temptab["Length"].loc[index] = length / 1000

#Figure 1A
Fig1A = (ggplot(Plastab) + 
         aes(x="Length", fill="predicted_mobility") + 
         geom_histogram(binwidth=10) + labs(x="Length (kb)", y='Plasmids', fill='Predicted Mobility') + #Change bins=15 to binwidth=10 05/06/23
         scale_fill_manual(values=['#117733', '#88CCEE', '#DDCC77'], labels=['Conjugative', 'Mobilizable', 'Non-Mobilizable']) + 
         theme_classic() +
         theme(axis_text=element_text(size=10), axis_title=element_text(size=12), legend_title=element_text(size=12), legend_text=element_text(size=10))
)
ggplot.save(Fig1A, filename='Length_histF1A.pdf', dpi=300)

#Figure 1B
Fig1B = (ggplot(Plastab) + 
         aes(x="GC", fill="predicted_mobility") + 
         geom_histogram(bins=12) + labs(x="GC content (%)", y='Plasmids', fill='Predicted Mobility') + 
         scale_fill_manual(values=['#117733', '#88CCEE', '#DDCC77'], labels=['Conjugative', 'Mobilizable', 'Non-Mobilizable']) + 
         theme_classic() +
         theme(axis_text=element_text(size=10), axis_title=element_text(size=12), legend_title=element_text(size=12), legend_text=element_text(size=10))
)
ggplot.save(Fig1B, filename='GC_histF1B.pdf', dpi=300)



Merging Incompatibility type results with rest of table to make Figure 2A

In [ ]:
#Re running imports and file loading
import pandas as pd
import numpy as np
import datetime as dt
from tqdm import tqdm

# Input the folder path (make sure it is r'path/') and filenames (due to changes made outside of python code, date numbers in names might not match with other cells)
folderPath = r'/workdir/users/hgl28/CDC_Isolates/Post-Dec2022/'
Old_results = 'Plasmid_Tab_withHits_2023-01-09_sed.csv'

#Read results
Old = pd.read_csv(folderPath + Old_results, delimiter=',', header=0)
Oldinc = Old[["Plasmid_ID", "Inc_Type"]].copy()
print(len(Oldinc))
oldnew = Oldinc["Inc_Type"].str.split(" ", n=0, expand=True)

#Making tidy version of Inc data in table
Oldinc["Inc1"] = oldnew[0]
Oldinc["Inc2"] = oldnew[1]
Oldinc["Inc3"] = oldnew[2]
Oldinc["Inc4"] = oldnew[3]
Oldinc["Inc5"] = oldnew[4]
Oldinc["Inc6"] = oldnew[5]
Oldinc["Inc7"] = oldnew[6]
Oldinc["Inc8"] = oldnew[7]

Oldinc.drop(columns=["Inc_Type"], inplace=True)

tempinc1 = Oldinc[["Plasmid_ID", "Inc1"]].copy()
tempinc1.columns = ["Plasmid_ID", "Inc_Type"]
tempinc2 = Oldinc[["Plasmid_ID", "Inc2"]].copy()
tempinc2.columns = ["Plasmid_ID", "Inc_Type"]
tempinc3 = Oldinc[["Plasmid_ID", "Inc3"]].copy()
tempinc3.columns = ["Plasmid_ID", "Inc_Type"]
tempinc4 = Oldinc[["Plasmid_ID", "Inc4"]].copy()
tempinc4.columns = ["Plasmid_ID", "Inc_Type"]
tempinc5 = Oldinc[["Plasmid_ID", "Inc5"]].copy()
tempinc5.columns = ["Plasmid_ID", "Inc_Type"]
tempinc6 = Oldinc[["Plasmid_ID", "Inc6"]].copy()
tempinc6.columns = ["Plasmid_ID", "Inc_Type"]
tempinc7 = Oldinc[["Plasmid_ID", "Inc7"]].copy()
tempinc7.columns = ["Plasmid_ID", "Inc_Type"]
tempinc8 = Oldinc[["Plasmid_ID", "Inc8"]].copy()
tempinc8.columns = ["Plasmid_ID", "Inc_Type"]

OldNewInctab = pd.concat([tempinc1, tempinc2, tempinc3, tempinc4, tempinc5, tempinc6, tempinc7, tempinc8])
OldNewInctab = OldNewInctab.dropna()
print(len(OldNewInctab))

#Getting Inc type counts
Mergeymerge = Plastab.merge(OldNewInctab, how='outer', on='Plasmid_ID')
NewIncPlastab = Mergeymerge.dropna(subset=["Inc_Type"])
NewIncPlastabtest = NewIncPlastab[NewIncPlastab["Inc_Type"] != '']
NewIncPlastabtest['Inc_Type_count'] = NewIncPlastabtest.groupby('Inc_Type')['Inc_Type'].transform('count')
mobconjNewPlas = NewIncPlastabtest[NewIncPlastabtest["predicted_mobility"] != "non-mobilizable"]
mobconjNewPlas['Inc_Type_count'] = mobconjNewPlas.groupby('Inc_Type')['Inc_Type'].transform('count')

#Figure 2A
Fig2A = (ggplot(mobconjNewPlas) + 
         aes(x='reorder(Inc_Type, Inc_Type_count)', fill='predicted_mobility') + 
         coord_flip() + 
         labs(x='Incompatibility Type', y='Plasmids', fill='Predicted Mobility') + 
         geom_bar() + 
         theme_classic() + 
         scale_fill_manual(['#117733', '#88CCEE'], labels=['Conjugative', 'Mobilizable']) + 
         theme(axis_text=element_text(size=10), axis_title=element_text(size=12), legend_title=element_text(size=12), legend_text=element_text(size=10), figure_size=(10,8))
)
ggplot.save(Fig2A, filename='Inc_F2A.pdf', dpi=300)


Making Figures 2B and 2C

In [ ]:
#Re running imports and file loading
import pandas as pd
import numpy as np
import datetime as dt
from tqdm import tqdm

# Input the folder path (make sure it is r'path/') and filenames (due to changes made outside of python code, date numbers in names might not match with other cells)
folderPath = r'/workdir/users/hgl28/CDC_Isolates/Post-Dec2022/'
Plasfile = 'Plasmid_TabMPS_withHits_2023-02-07_14-11-15.csv'
Plastab = pd.read_csv(folderPath + Plasfile, delimiter=',', header=0)

#Tidy version of relaxase data
relaxtab = Plastab[["Plasmid_ID", "relaxase_type(s)"]].copy()
relaxnew = relaxtab["relaxase_type(s)"].str.split(",", n=0, expand=True)

relaxtab["Mob1"] = relaxnew[0]
relaxtab["Mob2"] = relaxnew[1]
relaxtab["Mob3"] = relaxnew[2]

relaxtab.drop(columns=["relaxase_type(s)"], inplace=True)

tempmob1 = relaxtab[["Plasmid_ID", "Mob1"]].copy()
tempmob1.columns = ["Plasmid_ID", "relaxase_type"]
tempmob2 = relaxtab[["Plasmid_ID", "Mob2"]].copy()
tempmob2.columns = ["Plasmid_ID", "relaxase_type"]
tempmob3 = relaxtab[["Plasmid_ID", "Mob3"]].copy()
tempmob3.columns = ["Plasmid_ID", "relaxase_type"]

NewMobtab = pd.concat([tempmob1, tempmob2, tempmob3,])
NewMobtab = NewMobtab.dropna()
MergeymergeMOB = Plastab.merge(NewMobtab, how='outer', on='Plasmid_ID')
NewMobPlastab = MergeymergeMOB.dropna(subset=["relaxase_type"])
NewMobPlastab.drop_duplicates(inplace=True)

#Figure 2B
Fig2B = (ggplot(NewMobPlastab) + 
         aes(x='relaxase_type', fill='relaxase_type') + 
         labs(x='MOB Type', y='Plasmids', fill='MOB Type') + 
         geom_bar(show_legend=False) + 
         theme_classic() + 
         scale_x_discrete(limits=["MOBF", "MOBP", "MOBH", "MOBC", "MOBQ", "MOBV"]) + 
         scale_fill_manual(values=['#E69F00', '#CC79A7', '#009E73', '#F0E442', '#0072B2', '#D55E00']) + 
         theme(axis_text=element_text(size=10), axis_title=element_text(size=12), legend_title=element_text(size=12), legend_text=element_text(size=10)))
Fig2B.save(filename='Mob_F2B.pdf', dpi=300)

#Table manipulation for Figure 2C
NewMPFPlastab = Plastab[Plastab["mpf_type"] != '-']
print("Plasmids with MPF_types, ", len(NewMPFPlastab))
ProtSec = NewMPFPlastab[NewMPFPlastab["predicted_mobility"] != 'conjugative']
trueFig2Ctab = NewMPFPlastab[NewMPFPlastab["predicted_mobility"] != 'non-mobilizable']
MPFF = trueFig2Ctab[trueFig2Ctab["mpf_type"].str.contains("MPF_F")]
MPFT = trueFig2Ctab[trueFig2Ctab["mpf_type"].str.contains("MPF_T")]
MPFI = trueFig2Ctab[trueFig2Ctab["mpf_type"].str.contains("MPF_I")]
MPFG = trueFig2Ctab[trueFig2Ctab["mpf_type"].str.contains("MPF_G")]
print("Plasmids with MPF_F systems, ", len(MPFF))
print("Plasmids with MPF_T systems, ", len(MPFT))
print("Plasmids with MPF_I systems, ", len(MPFI))
print("Plasmids with MPF_G systems, ", len(MPFG))

#Figure 2C
MPFtabnew = Plastab[Plastab["mpf_type"] != '-']
Fig2C = (ggplot(MPFtabnew) + 
         aes(x="mpf_type", fill="mpf_type") + 
         geom_bar(show_legend=False) + 
         labs(x="MPF Type", y="Plasmids", fill='MOB Type') + 
         scale_x_discrete(limits=['MPF_F', 'MPF_T', 'MPF_I', 'MPF_G']) + 
         scale_fill_manual(values=['#785EF0', '#FFB000', '#FE6100', '#DC267F']) + 
         theme_classic() + 
         theme(axis_text=element_text(size=10), axis_title=element_text(size=12), legend_title=element_text(size=12), legend_text=element_text(size=10)))
Fig2C.save(filename='MPF_F2C.pdf', dpi=300)

Data Analysis and Stats

In [ ]:
#Imports
import pandas as pd
from scipy import stats
import scikit_posthocs as sph

#Folder and filenames
folderPath = r'/workdir/users/hgl28/CDC_Isolates/Post-Dec2022/'
Plasfile = 'Plasmid_Tab_Full_02-10-23.csv'
Plastab = pd.read_csv(folderPath + Plasfile, delimiter=',', header=0)

#Subtables
conjtab = Plastab[Plastab["predicted_mobility"] == 'conjugative']
mobtab = Plastab[Plastab["predicted_mobility"] == 'mobilizable']
nonmobtab = Plastab[Plastab["predicted_mobility"] == 'non-mobilizable']

#Get easy results
print("Conjugative Plasmids, ", len(conjtab))
print("Mobilizable Plasmids, ", len(mobtab))
print("Non-mobilizable Plasmids, ", len(nonmobtab))
print(Plastab["Length"].min())
print(Plastab["Length"].max())
print(Plastab["Length"].median())

#Assign a number for predicted mobility
Plastab["mobility_num"] = ''

for i in Plastab.index:
    temprow = Plastab.loc[i]
    if temprow["predicted_mobility"] == 'non-mobilizable':
        Plastab.at[i, "mobility_num"] = 0
    elif temprow["predicted_mobility"] == 'mobilizable':
        Plastab.at[i, "mobility_num"] = 1
    elif temprow["predicted_mobility"] == 'conjugative':
        Plastab.at[i, "mobility_num"] = 2

#Kruskal-Wallis followed by Dunn's for Length and mobility
testtab = Plastab[["Length", "mobility_num"]].copy()
print("Length min, ", testtab["Length"].min())
print("Length max, ", testtab["Length"].max())
print("Length median, ", testtab["Length"].median())
print(stats.kruskal(testtab["Length"], testtab["mobility_num"]))
sph.posthoc_dunn(testtab, val_col="Length", group_col="mobility_num", p_adjust = 'bonferroni')

#Kruskal-Wallis followed by Dunn's for GC and mobility
testGC = Plastab[["GC", "mobility_num"]].copy()
print("GC min, ", testGC["GC"].min())
print("GC max, ", testGC["GC"].max())
print("GC median, ", testGC["GC"].median())
print(stats.kruskal(testGC["GC"], testGC["mobility_num"]))
sph.posthoc_dunn(testGC, val_col='GC', group_col='mobility_num', p_adjust='bonferroni')

#Tidy version of ARG data to get relationship between presence of ARGs and mobility
rgitab = Plastab[["Plasmid_ID", "ARGs"]].copy()
new = rgitab["ARGs"].str.lstrip(" ").str.split(" ", n=0, expand=True)
rgitab.drop(columns=["ARGs"], inplace=True)
pd_list = []

for i in range(new.columns.__len__()):
    rgitab["Inc{}".format(i+1)] = new[i]
    tempinc1 = rgitab[["Plasmid_ID", "Inc{}".format(i+1)]].copy()
    tempinc1.columns = ["Plasmid_ID", "ARGs"]
    pd_list.append(tempinc1)

rgibig = pd.concat(pd_list)
rgibig = rgibig.dropna()
droptab = Plastab.drop(columns='ARGs')
rgibig = rgibig.merge(droptab, how='right', on='Plasmid_ID')

#Assign numbers for stats
rgibig["mobility_num"] = ''
rgibig['ARGyesno'] = ''

for i in rgibig.index:
    temprow = rgibig.loc[i]
    if temprow["predicted_mobility"] == 'non-mobilizable':
        rgibig.at[i, "mobility_num"] = 0
    elif temprow["predicted_mobility"] == 'mobilizable':
        rgibig.at[i, "mobility_num"] = 1
    elif temprow["predicted_mobility"] == 'conjugative':
        rgibig.at[i, "mobility_num"] = 2

for i in rgibig.index:
    temprow = rgibig.loc[i]
    if temprow["ARGs"] == '':
        rgibig.at[i, "PlasmidARGcount"] = 0
        rgibig.at[i, "ARGyesno"] = 0
    else:
        rgibig.at[i, "ARGyesno"] = 1

#Kruskal-Wallis followed by Dunn's for ARGs and mobility
rgishort = rgibig[["Plasmid_ID", "PlasmidARGcount", "ARGyesno", "mobility_num"]].copy().drop_duplicates()
print(stats.kruskal(rgishort["ARGyesno"], rgishort["mobility_num"]))
sph.posthoc_dunn(rgishort, val_col="ARGyesno", group_col="mobility_num", p_adjust = 'bonferroni')


BLAST results processing for Toxin-Antitoxin Data

In [ ]:
#Imports
import pandas as pd
import numpy as np
import datetime as dt
from tqdm import tqdm
#Rerunning BLAST code for ToxAntitox again (really should have made this a callable function)
filename_tatBR = 'Toxin_Antitoxin/Toxin-AT_02-02-23.txt'
filenamePID = 'plasmidORF_IDs_12-14-22.txt'
filenamePLid = 'plasmidIDs_01-23-23.txt'
filenamePlOF = 'Plasmid_Tab_Full_02-10-23.csv'
filenameORF_OF = 'InterPlasTabs/RGI_tab_12-14-22.csv'

# Loading the Blast results files
psiBR = pd.read_csv(folderPath + filename_tatBR, delimiter='\t', header=None)
psiBRNames = psiBR.columns
psiBR = psiBR[psiBRNames[0:5]]
psiBR.columns = ["Query","Subject","PID","Qcov","Eval"]

#Loading Output Files
ORF_OF = pd.read_csv(folderPath + filenameORF_OF, delimiter=',', header=0)
Pl_OF = pd.read_csv(folderPath + filenamePlOF, delimiter=',', header=0)

# Loading the Plasmid ID files
PlID = pd.read_csv(folderPath + filenamePID, delimiter='\t', header=0)
PLasmidID = pd.read_csv(folderPath + filenamePLid, delimiter='\t', header=0)
PLasmidID = PLasmidID.drop_duplicates()

# Setting up the Thresholds
PIDT = 50
QcovT2 = 70
EvalT = 10e-5
counter = 0
totalPlIDs = len(PlID)
PsiBRIndexHit = []


#Add Psi Protein Families to Blast Results tables

#--------------------------------------------------------------------------------------
TAcc = pd.read_csv(folderPath + 'Toxin_Antitoxin/TADB_T_acc_sed.csv', delimiter=',',header=0)
TAcc.drop(columns="Antitoxin", inplace=True)
ATAcc = pd.read_csv(folderPath + 'Toxin_Antitoxin/TADB_AT_acc_sed.csv', delimiter=',', header=0)
ATAcc.drop(columns="Toxin", inplace=True)
psiBR1 = psiBR.merge(TAcc, how = 'left', left_on = 'Query', right_on = 'TTA_ID')
psiBR1.drop(columns="Organism", inplace=True)
psiBR = psiBR1.merge(ATAcc, how = 'left', left_on = 'Query', right_on = 'ATTA_ID')
psiBR.drop(columns="Organism", inplace=True)

#--------------------------------------------------------------------------------------
for i in tqdm(PlID["ORF_ID"]):
    ChkPsi = psiBR.Subject == i
    if ChkPsi.sum() > 0:
        tempDFPar = psiBR[ChkPsi]
        coverage, current_best_idx = 0, 0
        for iii in tempDFPar.index:
            tempRowPar = tempDFPar.loc[iii]
            if tempRowPar.PID >= PIDT and tempRowPar.Qcov >= QcovT2 and tempRowPar.Eval <= EvalT:
                if tempRowPar.Qcov >= coverage:
                    coverage = tempRowPar.Qcov
                    current_best_idx = iii
        PsiBRIndexHit = np.append(PsiBRIndexHit,current_best_idx)
    counter+=1
print("Toxin_AT Protein Blast Scanning Finished! You got " + str(len(PsiBRIndexHit)) + " hits out of " + str(totalPlIDs) + " ORFs.")
counter = 0

# Scan Hits and add to table

#--------------------------------------------------------------------------------------
PsiHits = psiBR.iloc[PsiBRIndexHit]
PsiHits.drop(columns=["Unnamed: 4_x", "Unnamed: 4_y"], inplace=True)
PsiOF = ORF_OF.merge(PsiHits, how='inner', left_on='ORF_ID',right_on='Subject')
PsiOF = PsiOF[["ORF_ID", "Toxin", "Antitoxin"]].copy().drop_duplicates()
print("Toxin_AT Proteins Added ", len(PsiOF))
now = dt.datetime.now()
dt_string = now.strftime("%Y-%m-%d_%H-%M-%S")
PsiOF.to_csv(folderPath + "Toxin_AT"+ dt_string + ".csv")
print("All done!")


#Adding stuff to plasmid table
Plastab = pd.read_csv(folderPath + 'Plasmid_tab2_01-09-23.csv', delimiter=',', header=0)
ORFtabTox = PsiOF.dropna(subset=["Toxin"])
ORFtabTox.drop(columns='Antitoxin', inplace=True)
ORFtabAntitox = PsiOF.dropna(subset=["Antitoxin"])
ORFtabAntitox.drop(columns='Toxin', inplace=True)
#Identify plasmid ORFs and replace empty cells with hits from ORFs in plasmid table
for i in tqdm(PLasmidID["Plasmid_ID"]):
    index = Plastab[Plastab.Plasmid_ID == i]
    Chk = ORFtabTox["ORF_ID"].str.contains(i)
    AntitoxChk = ORFtabAntitox["ORF_ID"].str.contains(i)
    toxmatch = ORFtabTox[Chk]
    antitoxmatch = ORFtabAntitox[AntitoxChk]
    for ii in toxmatch.index:
        temprow = toxmatch.loc[ii]
        #Tox
        NewARG1 = str(temprow[1])
        tempstr = Plastab.at[Plastab[Plastab.Plasmid_ID == i].index[0],"Toxin"]
        temptype = type(Plastab.at[Plastab[Plastab.Plasmid_ID == i].index[0],"Toxin"])
        if temptype == str:
            Plastab.at[Plastab[Plastab.Plasmid_ID == i].index[0],"Toxin"] = tempstr + " " + NewARG1
        else:
            Plastab.at[Plastab[Plastab.Plasmid_ID == i].index[0],"Toxin"] = NewARG1
    for ii in antitoxmatch.index:
        temprow = antitoxmatch.loc[ii]
        #AntiTox
        NewARG2 = str(temprow[1])
        tempstr = Plastab.at[Plastab[Plastab.Plasmid_ID == i].index[0],"Antitoxin"]
        temptype = type(Plastab.at[Plastab[Plastab.Plasmid_ID == i].index[0],"Antitoxin"])
        if temptype == str:
            Plastab.at[Plastab[Plastab.Plasmid_ID == i].index[0],"Antitoxin"] = tempstr + " " + NewARG2
        else:
            Plastab.at[Plastab[Plastab.Plasmid_ID == i].index[0],"Antitoxin"] = NewARG2


#Finishing touches and output  
Plastab.drop(columns=["Length", "GC"], inplace=True)  
Psi_Final = Pl_OF.merge(Plastab, how='inner', on='Plasmid_ID')
Psi_Final = Psi_Final[["Genus","Organism","AR_Bank_ID","Plasmid_ID","Length","GC","ARGs","rep_type(s)","relaxase_type(s)","mpf_type","orit_type(s)","predicted_mobility","predicted_host_range_overall_rank","Virulence","Restriction Enzymes","Methylases", "Par","Psi","MPS","Toxin","Antitoxin", "RNA_Antitoxins"]].copy().drop_duplicates()
now = dt.datetime.now()
dt_string = now.strftime("%Y-%m-%d_%H-%M-%S")
Psi_Final.to_csv(folderPath + "Plasmid_TabToxin_AT_withHits_"+ dt_string + ".csv")
print("Everything Finished")

Making upsetplot (Figure 3)

In [ ]:
#Importing
import pandas as pd
import numpy as np
import datetime as dt
from tqdm import tqdm
from upsetplot import *
import matplotlib.pyplot as plt

folderPath = r'/workdir/users/hgl28/CDC_Isolates/Post-Dec2022/'
Plasfile = 'Plasmid_TabToxin_AT_withHits_2023-02-20_19-33-37.csv'
Plastab = pd.read_csv(folderPath + Plasfile, delimiter=',', header=0)
mobconjtab = Plastab[Plastab["predicted_mobility"] != 'non-mobilizable']
upsettab = mobconjtab[["Plasmid_ID", "Methylases", "Par", "Toxin", "Antitoxin", "RNA_Antitoxins", "Psi", "MPS"]].copy()
upsettab = upsettab.replace(r'^\s+$', np.nan, regex=True)
upsetfinal = pd.DataFrame(columns = ["Plasmid_ID","Methyltransferases", "Partitioning", "Toxin-Antitoxin", "SOS Inhibition", "Mating-Pair Stabilization"])
upsetfinal["Plasmid_ID"] = upsettab["Plasmid_ID"]
print(len(upsettab))
#Methylases
for i in tqdm(upsettab.index):
    temprow = upsettab.loc[i]
    tempid = temprow["Plasmid_ID"]
    temptype = type(temprow["Methylases"])
    if temptype != str:
        upsetfinal.at[upsetfinal[upsetfinal["Plasmid_ID"] == tempid].index[0], "Methyltransferases"] = 'False'
    else:
        upsetfinal.at[upsetfinal[upsetfinal["Plasmid_ID"] == tempid].index[0], "Methyltransferases"] = 'True'

#Partitioning
for i in tqdm(upsettab.index):
    temprow = upsettab.loc[i]
    tempid = temprow["Plasmid_ID"]
    temptype = type(temprow["Par"])
    if temptype != str:
        upsetfinal.at[upsetfinal[upsetfinal["Plasmid_ID"] == tempid].index[0], "Partitioning"] = 'False'
    else:
        upsetfinal.at[upsetfinal[upsetfinal["Plasmid_ID"] == tempid].index[0], "Partitioning"] = 'True'

#Toxin-Antitoxin
for i in tqdm(upsettab.index):
    temprow = upsettab.loc[i]
    tempid = temprow["Plasmid_ID"]
    temptypeT = type(temprow["Toxin"])
    temptypeAT = type(temprow["Antitoxin"])
    temptypeATR = type(temprow["RNA_Antitoxins"])
    if temptypeT != str:
        upsetfinal.at[upsetfinal[upsetfinal["Plasmid_ID"] == tempid].index[0], "Toxin-Antitoxin"] = 'False'
    elif temptypeAT != str and temptypeATR != str:
        upsetfinal.at[upsetfinal[upsetfinal["Plasmid_ID"] == tempid].index[0], "Toxin-Antitoxin"] = 'False'
    else:
        upsetfinal.at[upsetfinal[upsetfinal["Plasmid_ID"] == tempid].index[0], "Toxin-Antitoxin"] = 'True'

#SOS Inhibition
for i in tqdm(upsettab.index):
    temprow = upsettab.loc[i]
    tempid = temprow["Plasmid_ID"]
    temptype = type(temprow["Psi"])
    if temptype != str:
        upsetfinal.at[upsetfinal[upsetfinal["Plasmid_ID"] == tempid].index[0], "SOS Inhibition"] = 'False'
    else:
        upsetfinal.at[upsetfinal[upsetfinal["Plasmid_ID"] == tempid].index[0], "SOS Inhibition"] = 'True'

#MPS
for i in tqdm(upsettab.index):
    temprow = upsettab.loc[i]
    tempid = temprow["Plasmid_ID"]
    temptype = type(temprow["MPS"])
    if temptype != str:
        upsetfinal.at[upsetfinal[upsetfinal["Plasmid_ID"] == tempid].index[0], "Mating-Pair Stabilization"] = 'False'
    else:
        upsetfinal.at[upsetfinal[upsetfinal["Plasmid_ID"] == tempid].index[0], "Mating-Pair Stabilization"] = 'True'

upsetfinal = upsetfinal[["Plasmid_ID","Methyltransferases","Partitioning","Toxin-Antitoxin","SOS Inhibition","Mating-Pair Stabilization"]].copy()
print(len(upsetfinal))
for i in upsetfinal.index:
    temprow = upsetfinal.loc[i]
    if temprow["Methyltransferases"] == 'False' and temprow["Partitioning"] == 'False' and temprow["Toxin-Antitoxin"] == 'False' and temprow["SOS Inhibition"] == 'False' and temprow["Mating-Pair Stabilization"] == 'False':
        upsetfinal.drop(labels=i, inplace=True)

upsetfinal["Methyltransferases"]= upsetfinal["Methyltransferases"].apply(lambda x: True if x == "True" else False)
upsetfinal["Partitioning"]= upsetfinal["Partitioning"].apply(lambda x: True if x == "True" else False)
upsetfinal["Toxin-Antitoxin"]= upsetfinal["Toxin-Antitoxin"].apply(lambda x: True if x == "True" else False)
upsetfinal["SOS Inhibition"]= upsetfinal["SOS Inhibition"].apply(lambda x: True if x == "True" else False)
upsetfinal["Mating-Pair Stabilization"]= upsetfinal["Mating-Pair Stabilization"].apply(lambda x: True if x == "True" else False)


print(len(upsetfinal))


#Figure 3
plot(upsetfinal, show_counts='{:d}', min_degree=3)
plt.savefig('Fig3_upset_TATfixed.pdf', dpi=300) #I had made a previous version of this file with incorrect toxin-antitoxin data. This notebook only contains the correct code.

Making Supplementary Figure 1

In [ ]:
#Imports
import pandas as pd
from plotnine import *

#Folder and filenames and reading results
folderPath = r'/workdir/users/hgl28/CDC_Isolates/Post-Dec2022/'
Plasfile = 'Plasmid_TabToxin_AT_withHits_2023-02-20_19-33-37.csv'
RGIfile = 'RGI_BLAST_results/rgi_plasmidORFs.tsv'
Plastab = pd.read_csv(folderPath + Plasfile, delimiter=',', header=0)
RGIresults = pd.read_csv(folderPath + RGIfile, delimiter='\t', header=0)

#Tidy version of ARG data
rgitab = Plastab[["Plasmid_ID", "ARGs"]].copy()
new = rgitab["ARGs"].str.lstrip(" ").str.split(" ", n=0, expand=True)
rgitab.drop(columns=["ARGs"], inplace=True)
pd_list = []

for i in range(new.columns.__len__()):
    rgitab["Inc{}".format(i+1)] = new[i]
    tempinc1 = rgitab[["Plasmid_ID", "Inc{}".format(i+1)]].copy()
    tempinc1.columns = ["Plasmid_ID", "ARGs"]
    pd_list.append(tempinc1)

rgibig = pd.concat(pd_list)
rgibig = rgibig.dropna()
droptab = Plastab.drop(columns='ARGs')
rgibig = rgibig.merge(droptab, how='right', on='Plasmid_ID')
rgibig["Drug_Class"] = ''

#Getting drug class information for problematic drug class names
for i in rgibig.index:
    hitindex = RGIresults["ORF_ID"].str.contains(rgibig.at[i,"Plasmid_ID"])
    hits = RGIresults[hitindex]
    if len(hits) > 0:
        for ii in hits.index:
            tempARG = pd.Series(rgibig.at[i, "ARGs"])
            check = tempARG.isin([hits.at[ii, "Best_Hit_ARO"]])
            if check[0] == True:
                rgibig.at[i, "Drug_Class"] = hits.at[ii, "Drug Class"]
            elif tempARG[0] == 'catII_K12':
                rgibig.at[i, "Drug_Class"] = 'phenicol antibiotic'
            elif tempARG[0] == 'PC1_beta-lactamase_(blaZ)':
                rgibig.at[i, "Drug_Class"] = 'penam'
            elif tempARG[0] == 'S.aureus_mupA':
                rgibig.at[i, "Drug_Class"] = 'mupirocin'

#Getting drug class information for everything else
for i in rgibig.index:
    if rgibig.at[i, "Drug_Class"] == 'aminoglycoside antibiotic':
        rgibig.at[i, "Drug_Class"] = 'Aminoglycosides'
    elif rgibig.at[i, "Drug_Class"] == 'sulfonamide antibiotic':
        rgibig.at[i, "Drug_Class"] = 'Sulfonamides'
    elif rgibig.at[i, "Drug_Class"] == 'disinfecting agents and antiseptics':
        rgibig.at[i, "Drug_Class"] = 'Antiseptics'
    elif rgibig.at[i, "Drug_Class"] == 'monobactam; cephalosporin; penam; penem':
        rgibig.at[i, "Drug_Class"] = 'Beta-lactams'
    elif rgibig.at[i, "Drug_Class"] == 'carbapenem; cephalosporin; penam':
        rgibig.at[i, "Drug_Class"] = 'Beta-lactams'
    elif rgibig.at[i, "Drug_Class"] == 'diaminopyrimidine antibiotic':
        rgibig.at[i, "Drug_Class"] = 'Diaminopyrimidines'
    elif rgibig.at[i, "Drug_Class"] == 'tetracycline antibiotic':
        rgibig.at[i, "Drug_Class"] = 'Tetracyclines'
    elif rgibig.at[i, "Drug_Class"] == 'macrolide antibiotic':
        rgibig.at[i, "Drug_Class"] = 'Macrolides'
    elif rgibig.at[i, "Drug_Class"] == 'phenicol antibiotic':
        rgibig.at[i, "Drug_Class"] = 'Phenicols'
    elif rgibig.at[i, "Drug_Class"] == 'carbapenem; cephalosporin; cephamycin; penam':
        rgibig.at[i, "Drug_Class"] = 'Beta-lactams'
    elif rgibig.at[i, "Drug_Class"] == 'monobactam; carbapenem; cephalosporin; penam':
        rgibig.at[i, "Drug_Class"] = 'Beta-lactams'
    elif rgibig.at[i, "Drug_Class"] == 'glycopeptide antibiotic':
        rgibig.at[i, "Drug_Class"] = 'Glycopeptides'
    elif rgibig.at[i, "Drug_Class"] == 'cephalosporin; penam':
        rgibig.at[i, "Drug_Class"] = 'Beta-lactams'
    elif rgibig.at[i, "Drug_Class"] == 'fluoroquinolone antibiotic':
        rgibig.at[i, "Drug_Class"] = 'Fluoroquinolones'
    elif rgibig.at[i, "Drug_Class"] == 'fluoroquinolone antibiotic; aminoglycoside antibiotic':
        rgibig.at[i, "Drug_Class"] = 'Aminoglycosides'
    elif rgibig.at[i, "Drug_Class"] == 'penam':
        rgibig.at[i, "Drug_Class"] = 'Beta-lactams'
    elif rgibig.at[i, "Drug_Class"] == 'rifamycin antibiotic':
        rgibig.at[i, "Drug_Class"] = 'Rifamycins'
    elif rgibig.at[i, "Drug_Class"] == 'macrolide antibiotic; streptogramin antibiotic':
        rgibig.at[i, "Drug_Class"] = 'Macrolides'
    elif rgibig.at[i, "Drug_Class"] == 'cephamycin':
        rgibig.at[i, "Drug_Class"] = 'Beta-lactams'
    elif rgibig.at[i, "Drug_Class"] == 'cephalosporin; cephamycin':
        rgibig.at[i, "Drug_Class"] = 'Beta-lactams'
    elif rgibig.at[i, "Drug_Class"] == 'carbapenem; cephalosporin; cephamycin; penam; penem':
        rgibig.at[i, "Drug_Class"] = 'Beta-lactams'
    elif rgibig.at[i, "Drug_Class"] == 'macrolide antibiotic; lincosamide antibiotic; streptogramin antibiotic':
        rgibig.at[i, "Drug_Class"] = 'Macrolides'
    elif rgibig.at[i, "Drug_Class"] == 'macrolide antibiotic; lincosamide antibiotic; streptogramin antibiotic; streptogramin A antibiotic; streptogramin B antibiotic':
        rgibig.at[i, "Drug_Class"] = 'Macrolides'
    elif rgibig.at[i, "Drug_Class"] == 'macrolide antibiotic; streptogramin antibiotic; streptogramin B antibiotic':
        rgibig.at[i, "Drug_Class"] = 'Macrolides'
    elif rgibig.at[i, "Drug_Class"] == 'nucleoside antibiotic':
        rgibig.at[i, "Drug_Class"] = 'Nucleosides'
    elif rgibig.at[i, "Drug_Class"] == 'cephalosporin':
        rgibig.at[i, "Drug_Class"] = 'Beta-lactams'
    elif rgibig.at[i, "Drug_Class"] == 'cephalosporin; penam; penem':
        rgibig.at[i, "Drug_Class"] = 'Beta-lactams'
    elif rgibig.at[i, "Drug_Class"] == 'peptide antibiotic':
        rgibig.at[i, "Drug_Class"] = 'Peptides'
    elif rgibig.at[i, "Drug_Class"] == 'monobactam; cephalosporin':
        rgibig.at[i, "Drug_Class"] = 'Beta-lactams'
    elif rgibig.at[i, "Drug_Class"] == 'fluoroquinolone antibiotic; tetracycline antibiotic':
        rgibig.at[i, "Drug_Class"] = 'Tetracyclines'
    elif rgibig.at[i, "Drug_Class"] == 'mupirocin':
        rgibig.at[i, "Drug_Class"] = 'Mupirocin'

#Counts for plotting
rgibig['PlasmidARGcount'] = rgibig.groupby('Plasmid_ID')['Plasmid_ID'].transform('count')
rgibig["DrugClassCount"] = rgibig.groupby('Drug_Class')['Drug_Class'].transform('count')
rginoz = rgibig[rgibig["Drug_Class"] != '']


#Supplementary Figure 1A
SF1A_ARGsclass = (ggplot(rginoz) + 
          aes(x='reorder(Drug_Class, DrugClassCount)', fill='predicted_mobility') + 
          geom_bar() + 
          labs(x='Resistance Class', y='Number of Genes', fill='Predicted Mobility') + 
          theme_classic() + 
          coord_flip() + 
          scale_fill_manual(values=['#117733', '#88CCEE', '#DDCC77'], labels=['Conjugative', 'Mobilizable', 'Non-Mobilizable']))
SF1A_ARGsclass.save(filename='SF1A_ARGclass.pdf', dpi=300)

#Supplementary Figure 1B
rgishort = rgibig[["Plasmid_ID", "PlasmidARGcount"]].copy().drop_duplicates()
argfigtabV2 = Plastab.merge(rgishort, how='left', on='Plasmid_ID')
SFGARGfigB = (ggplot(argfigtabV2) +
           aes(x='PlasmidARGcount', fill='predicted_mobility') + 
           geom_histogram(bins=12) + 
           labs(x='Number of ARGs', y='Plasmids', fill='Predicted Mobility') + 
           theme_classic() + 
           scale_fill_manual(values=['#117733', '#88CCEE', '#DDCC77'], labels=["Conjugative", "Mobilizable", "Non-Mobilizable"]))
SFGARGfigB.save(filename="SF1B_ARGcounts.pdf", dpi=300)